In [ ]:
#|default_exp optimizer.optimi
#|default_cls_lvl 2

In [ ]:
#|exporti
# Contains code from:
# optimi - MIT Licence - Copyright (c) 2023 Benjamin Warner
# fastai - Apache License 2.0 - Copyright (c) 2023 fast.ai

# optimi Optimizers
> Fast, modern, and low precision optimizers with full fastai compatibility

In [ ]:
#|export
from __future__ import annotations

from optimi import (Adam, adam, Adan, adan, Lion, lion, RAdam, radam,
                    Ranger, ranger, SGD, sgd, StableAdamW, stableadamw)

from torch.optim.optimizer import _default_to_fused_or_foreach

from fastxtend.optimizer.utils import FastaiOptimizerAdapter, _convert_params
from fastxtend.imports import *

In [ ]:
#|hide
from nbdev.showdoc import *

## Adam -

In [ ]:
#|exporti
class AdamOptimiOptimizer(Adam, FastaiOptimizerAdapter):
    """Adam optimizer. Optionally with decoupled weight decay (AdamW).

    Parameters
    ----------
    params
        Iterable of parameters to optimize or dicts defining parameter groups
    lr
        Learning rate
    betas
        Coefficients for gradient and squared gradient moving averages (default: (0.9, 0.99))
    weight_decay
        Weight decay coefficient. If `decouple_wd` and `decouple_lr` are False, applies L2 penalty (default: 0)
    eps
        Added to denominator to improve numerical stability (default: 1e-6)
    decouple_wd
        Apply decoupled weight decay instead of L2 penalty (default: False)
    decouple_lr
        Apply fully decoupled weight decay instead of L2 penalty (default: False)
    max_lr
        Maximum scheduled learning rate. Set if `lr` is not the maximum scheduled learning rate and `decouple_lr` is True (default: None)
    kahan_sum
        Enables Kahan summation for more accurate parameter updates when training in low precision (float16 or bfloat16). If unspecified, automatically applies for low precision parameters (default: None)
    foreach
        Enables the foreach implementation. If unspecified, tries to use foreach over for-loop implementation since it is significantly faster (default: None)
    """

    def __init__(
        self,
        params: Iterable[Tensor] | Iterable[dict],
        lr: float,
        mom: float = 0.9,
        sqr_mom: float = 0.99,
        wd: float = 0,
        eps: float = 1e-6,
        decouple_wd: bool = False,
        decouple_lr: bool = False,
        max_lr: float | None = None,
        kahan_sum: bool | None = None,
        foreach: bool | None = None,
    ):
        params = super().convert_params(params)

        if not 0.0 <= mom < 1.0:
            raise ValueError(f"Invalid momentum parameter: {mom=}")
        if not 0.0 <= sqr_mom< 1.0:
            raise ValueError(f"Invalid squared momentum parameter: {sqr_mom=}")
        if not 0.0 <= wd:
            raise ValueError(f"Invalid weight decay: {wd=}")
        if not 0.0 <= eps:
            raise ValueError(f"Invalid epsilon: {eps=}")

        defaults = dict(
            lr=lr,
            mom=mom,
            sqr_mom=sqr_mom,
            wd=wd,
            eps=eps,
            weight_decay=wd,
            decouple_wd=decouple_wd,
            decouple_lr=decouple_lr,
            max_lr=max_lr,
            kahan_sum=kahan_sum,
            foreach=foreach,
            gradient_release=False,
            setup=False,
        )
        super(Adam, self).__init__(params, defaults)

    def _init_group(
        self,
        group: dict[str, Any],
        params_wd: list[Tensor],
        params_no_wd: list[Tensor],
        grads_wd: list[Tensor],
        grads_no_wd: list[Tensor],
        exp_avgs_wd: list[Tensor],
        exp_avgs_no_wd: list[Tensor],
        exp_avg_sqs_wd: list[Tensor],
        exp_avg_sqs_no_wd: list[Tensor],
        kahan_comps_wd: list[Tensor],
        kahan_comps_no_wd: list[Tensor],
    ):
        for p in group["params"]:
            if p.grad is None:
                continue

            state = self.state[p]
            self._init_state(group, state, p)

            if state.get("do_wd", True):
                params_wd.append(p)
                grads_wd.append(p.grad)
                exp_avgs_wd.append(state["exp_avg"])
                exp_avg_sqs_wd.append(state["exp_avg_sq"])
                kahan_comps_wd.append(state["kahan_comp"])
            else:
                params_no_wd.append(p)
                grads_no_wd.append(p.grad)
                exp_avgs_no_wd.append(state["exp_avg"])
                exp_avg_sqs_no_wd.append(state["exp_avg_sq"])
                kahan_comps_no_wd.append(state["kahan_comp"])

        if not group["setup"]:
            group["setup"] = True
            group["step"] = torch.tensor(0, dtype=torch.int32)

            if group["foreach"] is None:
                _, group["foreach"] = _default_to_fused_or_foreach(params_wd + params_no_wd, False, False)


    @torch.no_grad()
    def step(self, closure: Callable | None = None, param: Tensor | None = None):
        loss = None
        if closure is not None and param is None:
            with torch.enable_grad():
                loss = closure()

        if param is None:
            for group in self.param_groups:
                params_wd, params_no_wd, grads_wd, grads_no_wd = [], [], [], []
                exp_avgs_wd, exp_avgs_no_wd, exp_avg_sqs_wd, exp_avg_sqs_no_wd = [], [], [], []
                kahan_comps_wd, kahan_comps_no_wd = [], []

                self._init_group(
                    group,
                    params_wd,
                    params_no_wd,
                    grads_wd,
                    grads_no_wd,
                    exp_avgs_wd,
                    exp_avgs_no_wd,
                    exp_avg_sqs_wd,
                    exp_avg_sqs_no_wd,
                    kahan_comps_wd,
                    kahan_comps_no_wd
                )

                # Call adam for parameters with weight decay
                if len(params_wd) > 0:
                    adam(
                        params=params_wd,
                        grads=grads_wd,
                        exp_avgs=exp_avgs_wd,
                        exp_avg_sqs=exp_avg_sqs_wd,
                        kahan_comps=kahan_comps_wd,
                        lr=group["lr"],
                        beta1=group["mom"],
                        beta2=group["sqr_mom"],
                        weight_decay=group["wd"],
                        eps=group["eps"],
                        step=group["step"],
                        decouple_wd=group["decouple_wd"],
                        decouple_lr=group["decouple_lr"],
                        max_lr=group["max_lr"],
                        kahan_sum=group["kahan_sum"],
                        foreach=group["foreach"],
                        gradient_release=False,
                        optimizer_accumulation=False,
                    )

                # Call adam for parameters without weight decay
                if len(params_no_wd) > 0:
                    adam(
                        params=params_no_wd,
                        grads=grads_no_wd,
                        exp_avgs=exp_avgs_no_wd,
                        exp_avg_sqs=exp_avg_sqs_no_wd,
                        kahan_comps=kahan_comps_no_wd,
                        lr=group["lr"],
                        beta1=group["mom"],
                        beta2=group["sqr_mom"],
                        weight_decay=0,
                        eps=group["eps"],
                        step=group["step"],
                        decouple_wd=group["decouple_wd"],
                        decouple_lr=group["decouple_lr"],
                        max_lr=group["max_lr"],
                        kahan_sum=group["kahan_sum"],
                        foreach=group["foreach"],
                        gradient_release=False,
                        optimizer_accumulation=False,
                    )
        else:
            state = self.state[param]
            group = state["group"]
            self._init_state(group, state, param)

            adam(
                params=param,
                grads=param.grad,
                exp_avgs=state["exp_avg"],
                exp_avg_sqs=state["exp_avg_sq"],
                kahan_comps=state["kahan_comp"],
                lr=group["lr"],
                beta1=group["mom"],
                beta2=group["sqr_mom"],
                weight_decay=group["wd"] if state.get("do_wd", True) else 0,
                eps=group["eps"],
                step=state["step"],
                decouple_wd=group["decouple_wd"],
                decouple_lr=group["decouple_lr"],
                max_lr=group["max_lr"],
                kahan_sum=group["kahan_sum"],
                foreach=False,
                gradient_release=True,
                optimizer_accumulation=self._optimizer_accumulation,
            )

        return loss


In [ ]:
show_doc(AdamOptimiOptimizer)

## AdamW -

In [ ]:
#|exporti
class AdamWOptimiOptimizer(AdamOptimiOptimizer):
    """AdamW optimizer. Adam with decoupled weight decay.

    Parameters
    ----------
    params
        Iterable of parameters to optimize or dicts defining parameter groups
    lr
        Learning rate
    betas
        Coefficients for gradient and squared gradient moving averages (default: (0.9, 0.99))
    weight_decay
        Weight decay coefficient. If `decouple_wd` and `decouple_lr` are False, applies L2 penalty (default: 0)
    eps
        Added to denominator to improve numerical stability (default: 1e-6)
    decouple_lr
        Apply fully decoupled weight decay instead of L2 penalty (default: False)
    max_lr
        Maximum scheduled learning rate. Set if `lr` is not the maximum scheduled learning rate and `decouple_lr` is True (default: None)
    kahan_sum
        Enables Kahan summation for more accurate parameter updates when training in low precision (float16 or bfloat16). If unspecified, automatically applies for low precision parameters (default: None)
    foreach
        Enables the foreach implementation. If unspecified, tries to use foreach over for-loop implementation since it is significantly faster (default: None)
    """

    def __init__(
        self,
        params: Iterable[Tensor] | Iterable[dict],
        lr: float,
        mom: float = 0.9,
        sqr_mom: float = 0.99,
        wd: float = 1e-2,
        eps: float = 1e-6,
        decouple_lr: bool = False,
        max_lr: float | None = None,
        kahan_sum: bool | None = None,
        foreach: bool | None = None,
    ):
        super().__init__(
            params=params,
            lr=lr,
            mom=mom,
            sqr_mom=sqr_mom,
            wd=wd,
            eps=eps,
            decouple_wd=True,
            decouple_lr=decouple_lr,
            max_lr=max_lr,
            kahan_sum=kahan_sum,
            foreach=foreach,
        )


In [ ]:
show_doc(AdamWOptimiOptimizer)

## Adan -

In [ ]:
#|exporti
class AdanOptimiOptimizer(Adan, FastaiOptimizerAdapter):
    """Adan Optimizer: Adaptive Nesterov Momentum Algorithm.

    Parameters
    ----------
    params
        Iterable of parameters to optimize or dicts defining parameter groups
    lr
        Learning rate
    betas
        Coefficients for gradient, gradient difference, and squared gradient moving averages
    weight_decay
        Weight decay coefficient. If `decouple_lr` is False, applies decoupled weight decay
    eps
        Added to denominator to improve numerical stability
    decouple_lr
        Apply fully decoupled weight decay instead of decoupled weight decay
    max_lr
        Maximum scheduled learning rate. Set if `lr` is not the maximum scheduled learning rate and `decouple_lr` is True
    adam_wd
        Apply weight decay before parameter update (Adam-style), instead of after the update per Adan algorithm
    kahan_sum
        Enables Kahan summation for more accurate parameter updates when training in low precision (float16 or bfloat16). If unspecified, automatically applies for low precision parameters
    foreach
        Enables the foreach implementation. If unspecified, tries to use foreach over for-loop implementation since it is significantly faster
    """

    def __init__(
        self,
        params: Iterable[Tensor] | Iterable[dict],
        lr: float,
        beta1: float = 0.98,
        beta2: float = 0.92,
        beta3: float = 0.99,
        wd: float = 2e-2,
        eps: float = 1e-6,
        decouple_lr: bool = False,
        max_lr: float | None = None,
        adam_wd: bool = False,
        kahan_sum: bool | None = False,
        foreach: bool | None = None,
    ):
        params = super().convert_params(params)

        if not 0.0 <= beta1 < 1.0:
            raise ValueError(f"Invalid beta1 parameter: {beta1=}")
        if not 0.0 <= beta2 < 1.0:
            raise ValueError(f"Invalid beta2 parameter: {beta2=}")
        if not 0.0 <= beta3 < 1.0:
            raise ValueError(f"Invalid beta3 parameter: {beta3=}")
        if not 0.0 <= wd:
            raise ValueError(f"Invalid weight decay: {wd=}")
        if not 0.0 <= eps:
            raise ValueError(f"Invalid epsilon: {eps=}")

        defaults = dict(
            lr=lr,
            beta1=beta1,
            beta2=beta2,
            beta3=beta3,
            wd=wd,
            eps=eps,
            weight_decay=wd,
            decouple_lr=decouple_lr,
            max_lr=max_lr,
            kahan_sum=kahan_sum,
            foreach=foreach,
            adam_wd=adam_wd,
            gradient_release=False,
            setup=False,
        )
        super(Adan, self).__init__(params, defaults)

    def _init_group(
        self,
        group: dict[str, Any],
        params_wd: list[Tensor],
        params_no_wd: list[Tensor],
        grads_wd: list[Tensor],
        grads_no_wd: list[Tensor],
        exp_avgs_wd: list[Tensor],
        exp_avgs_no_wd: list[Tensor],
        exp_avg_diffs_wd: list[Tensor],
        exp_avg_diffs_no_wd: list[Tensor],
        exp_avg_sqs_wd: list[Tensor],
        exp_avg_sqs_no_wd: list[Tensor],
        prev_grads_wd: list[Tensor],
        prev_grads_no_wd: list[Tensor],
        kahan_comps_wd: list[Tensor],
        kahan_comps_no_wd: list[Tensor],
    ):
        for p in group["params"]:
            if p.grad is None:
                continue

            state = self.state[p]
            self._init_state(group, state, p)

            if state.get("do_wd", False):
                params_wd.append(p)
                grads_wd.append(p.grad)
                exp_avgs_wd.append(state["exp_avg"])
                exp_avg_diffs_wd.append(state["exp_avg_diff"])
                exp_avg_sqs_wd.append(state["exp_avg_sq"])
                prev_grads_wd.append(state["prev_grad"])
                kahan_comps_wd.append(state["kahan_comp"])
            else:
                params_no_wd.append(p)
                grads_no_wd.append(p.grad)
                exp_avgs_no_wd.append(state["exp_avg"])
                exp_avg_diffs_no_wd.append(state["exp_avg_diff"])
                exp_avg_sqs_no_wd.append(state["exp_avg_sq"])
                prev_grads_no_wd.append(state["prev_grad"])
                kahan_comps_no_wd.append(state["kahan_comp"])

        if not group["setup"]:
            group["setup"] = True
            group["step"] = torch.tensor(0, dtype=torch.int32)

            if group["foreach"] is None:
                _, group["foreach"] = _default_to_fused_or_foreach(params_wd + params_no_wd, False, False)
            if group["kahan_sum"]:
                group["adam_wd"] = False

    @torch.no_grad()
    def step(self, closure: Callable | None = None, param: Tensor | None = None):
        loss = None
        if closure is not None and param is None:
            with torch.enable_grad():
                loss = closure()

        if param is None:
            for group in self.param_groups:
                params_wd, params_no_wd, grads_wd, grads_no_wd = [], [], [], []
                exp_avgs_wd, exp_avgs_no_wd, exp_avg_diffs_wd, exp_avg_diffs_no_wd = [], [], [], []
                exp_avg_sqs_wd, exp_avg_sqs_no_wd, prev_grads_wd, prev_grads_no_wd = [], [], [], []
                kahan_comps_wd, kahan_comps_no_wd = [], []

                self._init_group(
                    group,
                    params_wd,
                    params_no_wd,
                    grads_wd,
                    grads_no_wd,
                    exp_avgs_wd,
                    exp_avgs_no_wd,
                    exp_avg_diffs_wd,
                    exp_avg_diffs_no_wd,
                    exp_avg_sqs_wd,
                    exp_avg_sqs_no_wd,
                    prev_grads_wd,
                    prev_grads_no_wd,
                    kahan_comps_wd,
                    kahan_comps_no_wd
                )

                # Call adan for parameters with weight decay
                if len(params_wd) > 0:
                    adan(
                        params=params_wd,
                        grads=grads_wd,
                        exp_avgs=exp_avgs_wd,
                        exp_avg_diffs=exp_avg_diffs_wd,
                        exp_avg_sqs=exp_avg_sqs_wd,
                        prev_grads=prev_grads_wd,
                        kahan_comps=kahan_comps_wd,
                        lr=group["lr"],
                        beta1=group["beta1"],
                        beta2=group["beta2"],
                        beta3=group["beta3"],
                        eps=group["eps"],
                        weight_decay=group["wd"],
                        step=group["step"],
                        decouple_lr=group["decouple_lr"],
                        max_lr=group["max_lr"],
                        kahan_sum=group["kahan_sum"],
                        foreach=group["foreach"],
                        gradient_release=False,
                        optimizer_accumulation=False,
                    )

                # Call adan for parameters without weight decay
                if len(params_no_wd) > 0:
                    adan(
                        params=params_no_wd,
                        grads=grads_no_wd,
                        exp_avgs=exp_avgs_no_wd,
                        exp_avg_diffs=exp_avg_diffs_no_wd,
                        exp_avg_sqs=exp_avg_sqs_no_wd,
                        prev_grads=prev_grads_no_wd,
                        kahan_comps=kahan_comps_no_wd,
                        lr=group["lr"],
                        beta1=group["beta1"],
                        beta2=group["beta2"],
                        beta3=group["beta3"],
                        eps=group["eps"],
                        weight_decay=0.0,
                        step=group["step"],
                        decouple_lr=group["decouple_lr"],
                        max_lr=group["max_lr"],
                        kahan_sum=group["kahan_sum"],
                        foreach=group["foreach"],
                        gradient_release=False,
                        optimizer_accumulation=False,
                    )
        else:
            state = self.state[param]
            group = state["group"]
            self._init_state(group, state, param)

            adan(
                params=[param],
                grads=[param.grad],
                exp_avgs=[state["exp_avg"]],
                exp_avg_diffs=[state["exp_avg_diff"]],
                exp_avg_sqs=[state["exp_avg_sq"]],
                prev_grads=[state["prev_grad"]],
                kahan_comps=[state["kahan_comp"]],
                lr=group["lr"],
                beta1=group["beta1"],
                beta2=group["beta2"],
                beta3=group["beta3"],
                eps=group["eps"],
                weight_decay=group["wd"] if state.get("do_wd", True) else 0,
                step=state["step"],
                decouple_lr=group["decouple_lr"],
                max_lr=group["max_lr"],
                kahan_sum=group["kahan_sum"],
                foreach=False,
                gradient_release=True,
                optimizer_accumulation=self._optimizer_accumulation,
            )

        return loss



In [ ]:
show_doc(AdanOptimiOptimizer)

## Lion -

In [ ]:
#|exporti
class LionOptimiOptimizer(Lion, FastaiOptimizerAdapter):
    """Lion optimizer. Evolved Sign Momentum.

    Parameters
    ----------
    params
        Iterable of parameters to optimize or dicts defining parameter groups
    lr
        Learning rate
    betas
        Coefficients for update moving average and gradient moving average (default: (0.9, 0.99))
    weight_decay
        Weight decay coefficient. If `decouple_lr` is False, applies decoupled weight decay (default: 0)
    decouple_lr
        Apply fully decoupled weight decay instead of decoupled weight decay (default: False)
    max_lr
        Maximum scheduled learning rate. Set if `lr` is not the maximum scheduled learning rate and `decouple_lr` is True (default: None)
    kahan_sum
        Enables Kahan summation for more accurate parameter updates when training in low precision (float16 or bfloat16). If unspecified, automatically applies for low precision parameters (default: None)
    foreach
        Enables the foreach implementation. If unspecified, tries to use foreach over for-loop implementation since it is significantly faster (default: None)
    """

    def __init__(
        self,
        params: Iterable[Tensor] | Iterable[dict],
        lr: float,
        beta1: float = 0.9,
        beta2: float = 0.99,
        wd: float = 0,
        decouple_lr: bool = False,
        max_lr: float | None = None,
        kahan_sum: bool | None = None,
        foreach: bool | None = None,
    ):
        params = super().convert_params(params)

        if not 0.0 <= beta1 < 1.0:
            raise ValueError(f"Invalid beta1 parameter: {beta1=}")
        if not 0.0 <= beta2 < 1.0:
            raise ValueError(f"Invalid beta2 parameter: {beta2=}")
        if not 0.0 <= wd:
            raise ValueError(f"Invalid weight decay: {wd=}")

        defaults = dict(
            lr=lr,
            beta1=beta1,
            beta2=beta2,
            wd=wd,
            weight_decay=wd,
            decouple_lr=decouple_lr,
            max_lr=max_lr,
            kahan_sum=kahan_sum,
            foreach=foreach,
            gradient_release=False,
            setup=False,
        )
        super(Lion, self).__init__(params, defaults)

    def _init_group(
        self,
        group: dict[str, Any],
        params_wd: list[Tensor],
        params_no_wd: list[Tensor],
        grads_wd: list[Tensor],
        grads_no_wd: list[Tensor],
        exp_avgs_wd: list[Tensor],
        exp_avgs_no_wd: list[Tensor],
        kahan_comps_wd: list[Tensor],
        kahan_comps_no_wd: list[Tensor],
    ):
        for p in group["params"]:
            if p.grad is None:
                continue

            state = self.state[p]
            self._init_state(group, state, p)

            if state.get("do_wd", True):
                params_wd.append(p)
                grads_wd.append(p.grad)
                exp_avgs_wd.append(state["exp_avg"])
                kahan_comps_wd.append(state["kahan_comp"])
            else:
                params_no_wd.append(p)
                grads_no_wd.append(p.grad)
                exp_avgs_no_wd.append(state["exp_avg"])
                kahan_comps_no_wd.append(state["kahan_comp"])

        if not group["setup"]:
            group["setup"] = True

            if group["foreach"] is None:
                _, group["foreach"] = _default_to_fused_or_foreach(params_wd + params_no_wd, False, False)

    @torch.no_grad()
    def step(self, closure: Callable | None = None, param: Tensor | None = None):
        loss = None
        if closure is not None and param is None:
            with torch.enable_grad():
                loss = closure()

        if param is None:
            for group in self.param_groups:
                params_wd, params_no_wd, grads_wd, grads_no_wd = [], [], [], []
                exp_avgs_wd, exp_avgs_no_wd, kahan_comps_wd, kahan_comps_no_wd = [], [], [], []

                self._init_group(
                    group,
                    params_wd,
                    params_no_wd,
                    grads_wd,
                    grads_no_wd,
                    exp_avgs_wd,
                    exp_avgs_no_wd,
                    kahan_comps_wd,
                    kahan_comps_no_wd
                )

                # Call lion for parameters with weight decay
                if len(params_wd) > 0:
                    lion(
                        params=params_wd,
                        grads=grads_wd,
                        exp_avgs=exp_avgs_wd,
                        kahan_comps=kahan_comps_wd,
                        lr=group["lr"],
                        beta1=group["beta1"],
                        beta2=group["beta2"],
                        weight_decay=group["wd"],
                        decouple_lr=group["decouple_lr"],
                        max_lr=group["max_lr"],
                        kahan_sum=group["kahan_sum"],
                        foreach=group["foreach"],
                        gradient_release=False,
                        optimizer_accumulation=False,
                    )

                # Call lion for parameters without weight decay
                if len(params_no_wd) > 0:
                    lion(
                        params=params_no_wd,
                        grads=grads_no_wd,
                        exp_avgs=exp_avgs_no_wd,
                        kahan_comps=kahan_comps_no_wd,
                        lr=group["lr"],
                        beta1=group["beta1"],
                        beta2=group["beta2"],
                        weight_decay=0,
                        decouple_lr=group["decouple_lr"],
                        max_lr=group["max_lr"],
                        kahan_sum=group["kahan_sum"],
                        foreach=group["foreach"],
                        gradient_release=False,
                        optimizer_accumulation=False,
                    )
        else:
            state = self.state[param]
            group = state["group"]
            self._init_state(group, state, param)

            lion(
                params=param,
                grads=param.grad,
                exp_avgs=state["exp_avg"],
                kahan_comps=state["kahan_comp"],
                lr=group["lr"],
                beta1=group["beta1"],
                beta2=group["beta2"],
                weight_decay=group["wd"] if state.get("do_wd", True) else 0,
                decouple_lr=group["decouple_lr"],
                max_lr=group["max_lr"],
                kahan_sum=group["kahan_sum"],
                foreach=False,
                gradient_release=True,
                optimizer_accumulation=self._optimizer_accumulation,
            )

        return loss

In [ ]:
show_doc(LionOptimiOptimizer)

## RAdam -

In [ ]:
#|exporti
class RAdamOptimiOptimizer(RAdam, FastaiOptimizerAdapter):
    """Rectified Adam optimizer. Optionally with decoupled weight decay.

    Parameters
    ----------
    params
        Iterable of parameters to optimize or dicts defining parameter groups
    lr
        Learning rate
    betas
        Coefficients for gradient and squared gradient moving averages (default: (0.9, 0.99))
    weight_decay
        Weight decay coefficient. If `decouple_wd` and `decouple_lr` are False, applies L2 penalty (default: 0)
    eps
        Added to denominator to improve numerical stability (default: 1e-6)
    decouple_wd
        Apply decoupled weight decay instead of L2 penalty (default: True)
    decouple_lr
        Apply fully decoupled weight decay instead of L2 penalty (default: False)
    max_lr
        Maximum scheduled learning rate. Set if `lr` is not the maximum scheduled learning rate and `decouple_lr` is True (default: None)
    kahan_sum
        Enables Kahan summation for more accurate parameter updates when training in low precision (float16 or bfloat16). If unspecified, automatically applies for low precision parameters (default: None)
    foreach
        Enables the foreach implementation. If unspecified, tries to use foreach over for-loop implementation since it is significantly faster (default: None)
    """

    def __init__(
        self,
        params: Iterable[Tensor] | Iterable[dict],
        lr: float,
        mom: float = 0.9,
        sqr_mom: float = 0.99,
        wd: float = 0,
        eps: float = 1e-6,
        decouple_wd: bool = False,
        decouple_lr: bool = False,
        max_lr: float | None = None,
        kahan_sum: bool | None = None,
        foreach: bool | None = None,
    ):
        params = super().convert_params(params)

        if not 0.0 <= mom < 1.0:
            raise ValueError(f"Invalid momentum parameter: {mom=}")
        if not 0.0 <= sqr_mom< 1.0:
            raise ValueError(f"Invalid squared momentum parameter: {sqr_mom=}")
        if not 0.0 <= wd:
            raise ValueError(f"Invalid weight decay: {wd=}")
        if not 0.0 <= eps:
            raise ValueError(f"Invalid epsilon: {eps=}")

        defaults = dict(
            lr=lr,
            mom=mom,
            sqr_mom=sqr_mom,
            eps=eps,
            wd=wd,
            weight_decay=wd,
            decouple_wd=decouple_wd,
            decouple_lr=decouple_lr,
            max_lr=max_lr,
            kahan_sum=kahan_sum,
            foreach=foreach,
            gradient_release=False,
            setup=False,
        )
        super(RAdam, self).__init__(params, defaults)

    def _init_group(
        self,
        group: dict[str, Any],
        params_wd: list[Tensor],
        params_no_wd: list[Tensor],
        grads_wd: list[Tensor],
        grads_no_wd: list[Tensor],
        exp_avgs_wd: list[Tensor],
        exp_avgs_no_wd: list[Tensor],
        exp_avg_sqs_wd: list[Tensor],
        exp_avg_sqs_no_wd: list[Tensor],
        kahan_comps_wd: list[Tensor],
        kahan_comps_no_wd: list[Tensor],
    ):
        for p in group["params"]:
            if p.grad is None:
                continue

            state = self.state[p]
            self._init_state(group, state, p)

            if state.get("do_wd", True):
                params_wd.append(p)
                grads_wd.append(p.grad)
                exp_avgs_wd.append(state["exp_avg"])
                exp_avg_sqs_wd.append(state["exp_avg_sq"])
                kahan_comps_wd.append(state["kahan_comp"])
            else:
                params_no_wd.append(p)
                grads_no_wd.append(p.grad)
                exp_avgs_no_wd.append(state["exp_avg"])
                exp_avg_sqs_no_wd.append(state["exp_avg_sq"])
                kahan_comps_no_wd.append(state["kahan_comp"])

        if not group["setup"]:
            group["setup"] = True
            group["step"] = torch.tensor(0, dtype=torch.int32)

            if group["foreach"] is None:
                _, group["foreach"] = _default_to_fused_or_foreach(params_wd + params_no_wd, False, False)

    @torch.no_grad()
    def step(self, closure: Callable | None = None, param: Tensor | None = None):
        loss = None
        if closure is not None and param is None:
            with torch.enable_grad():
                loss = closure()

        if param is None:
            for group in self.param_groups:
                params_wd, params_no_wd, grads_wd, grads_no_wd = [], [], [], []
                exp_avgs_wd, exp_avgs_no_wd, exp_avg_sqs_wd, exp_avg_sqs_no_wd = [], [], [], []
                kahan_comps_wd, kahan_comps_no_wd = [], []

                self._init_group(
                    group,
                    params_wd,
                    params_no_wd,
                    grads_wd,
                    grads_no_wd,
                    exp_avgs_wd,
                    exp_avgs_no_wd,
                    exp_avg_sqs_wd,
                    exp_avg_sqs_no_wd,
                    kahan_comps_wd,
                    kahan_comps_no_wd
                )

                # Call radam for parameters with weight decay
                if len(params_wd) > 0:
                    radam(
                        params=params_wd,
                        grads=grads_wd,
                        exp_avgs=exp_avgs_wd,
                        exp_avg_sqs=exp_avg_sqs_wd,
                        kahan_comps=kahan_comps_wd,
                        lr=group["lr"],
                        beta1=group["mom"],
                        beta2=group["sqr_mom"],
                        weight_decay=group["wd"],
                        eps=group["eps"],
                        step=group["step"],
                        decouple_wd=group["decouple_wd"],
                        decouple_lr=group["decouple_lr"],
                        max_lr=group["max_lr"],
                        kahan_sum=group["kahan_sum"],
                        foreach=group["foreach"],
                        gradient_release=False,
                        optimizer_accumulation=False,
                    )

                # Call radam for parameters without weight decay
                if len(params_no_wd) > 0:
                    radam(
                        params=params_no_wd,
                        grads=grads_no_wd,
                        exp_avgs=exp_avgs_no_wd,
                        exp_avg_sqs=exp_avg_sqs_no_wd,
                        kahan_comps=kahan_comps_no_wd,
                        lr=group["lr"],
                        beta1=group["mom"],
                        beta2=group["sqr_mom"],
                        weight_decay=0,
                        eps=group["eps"],
                        step=group["step"],
                        decouple_wd=group["decouple_wd"],
                        decouple_lr=group["decouple_lr"],
                        max_lr=group["max_lr"],
                        kahan_sum=group["kahan_sum"],
                        foreach=group["foreach"],
                        gradient_release=False,
                        optimizer_accumulation=False,
                    )
        else:
            state = self.state[param]
            group = state["group"]
            self._init_state(group, state, param)

            radam(
                params=param,
                grads=param.grad,
                exp_avgs=state["exp_avg"],
                exp_avg_sqs=state["exp_avg_sq"],
                kahan_comps=state["kahan_comp"],
                lr=group["lr"],
                beta1=group["mom"],
                beta2=group["sqr_mom"],
                weight_decay=group["wd"] if state.get("do_wd", True) else 0,
                eps=group["eps"],
                step=state["step"],
                decouple_wd=group["decouple_wd"],
                decouple_lr=group["decouple_lr"],
                max_lr=group["max_lr"],
                kahan_sum=group["kahan_sum"],
                foreach=False,
                gradient_release=True,
                optimizer_accumulation=self._optimizer_accumulation,
            )

        return loss

In [ ]:
show_doc(RAdamOptimiOptimizer)

## Ranger -

In [ ]:
#|exporti
class RangerOptimiOptimizer(Ranger, FastaiOptimizerAdapter):
    """Ranger optimizer. RAdam with Lookahead.

    Parameters
    ----------
    params
        Iterable of parameters to optimize or dicts defining parameter groups
    lr
        Learning rate
    betas
        Coefficients for gradient and squared gradient moving averages (default: (0.9, 0.99))
    weight_decay
        Weight decay coefficient. If `decouple_wd` and `decouple_lr` are False, applies L2 penalty (default: 0)
    eps
        Added to denominator to improve numerical stability (default: 1e-6)
    k
        Lookahead synchronization period (default: 6)
    alpha
        Lookahead weight interpolation coefficient (default: 0.5)
    decouple_wd
        Apply decoupled weight decay instead of L2 penalty (default: True)
    decouple_lr
        Apply fully decoupled weight decay instead of L2 penalty (default: False)
    max_lr
        Maximum scheduled learning rate. Set if `lr` is not the maximum scheduled learning rate and `decouple_lr` is True (default: None)
    kahan_sum
        Enables Kahan summation for more accurate parameter updates when training in low precision (float16 or bfloat16). If unspecified, automatically applies for low precision parameters (default: None)
    foreach
        Enables the foreach implementation. If unspecified, tries to use foreach over for-loop implementation since it is significantly faster (default: None)
    """

    def __init__(
        self,
        params: Iterable[Tensor] | Iterable[dict],
        lr: float,
        mom: float = 0.9,
        sqr_mom: float = 0.99,
        wd: float = 0,
        eps: float = 1e-6,
        k: int = 6,
        alpha: float = 0.5,
        decouple_wd: bool = True,
        decouple_lr: bool = False,
        max_lr: float | None = None,
        kahan_sum: bool | None = None,
        foreach: bool | None = None,
    ):
        params = super().convert_params(params)

        if not 0.0 <= mom < 1.0:
            raise ValueError(f"Invalid momentum parameter: {mom=}")
        if not 0.0 <= sqr_mom< 1.0:
            raise ValueError(f"Invalid squared momentum parameter: {sqr_mom=}")
        if not 0.0 <= wd:
            raise ValueError(f"Invalid weight decay: {wd=}")
        if not 0.0 <= eps:
            raise ValueError(f"Invalid epsilon: {eps=}")

        defaults = dict(
            lr=lr,
            mom=mom,
            sqr_mom=sqr_mom,
            wd=wd,
            eps=eps,
            k=k,
            alpha=alpha,
            weight_decay=wd,
            decouple_wd=decouple_wd,
            decouple_lr=decouple_lr,
            max_lr=max_lr,
            kahan_sum=kahan_sum,
            foreach=foreach,
            gradient_release=False,
            setup=False,
        )
        super(Ranger, self).__init__(params, defaults)

    def _init_group(
        self,
        group: dict[str, Any],
        params_wd: list[Tensor],
        params_no_wd: list[Tensor],
        grads_wd: list[Tensor],
        grads_no_wd: list[Tensor],
        exp_avgs_wd: list[Tensor],
        exp_avgs_no_wd: list[Tensor],
        exp_avg_sqs_wd: list[Tensor],
        exp_avg_sqs_no_wd: list[Tensor],
        la_params_wd: list[Tensor],
        la_params_no_wd: list[Tensor],
        kahan_comps_wd: list[Tensor],
        kahan_comps_no_wd: list[Tensor],
    ):
        for p in group["params"]:
            if p.grad is None:
                continue

            state = self.state[p]
            self._init_state(group, state, p)

            if state.get("do_wd", True):
                params_wd.append(p)
                grads_wd.append(p.grad)
                exp_avgs_wd.append(state["exp_avg"])
                exp_avg_sqs_wd.append(state["exp_avg_sq"])
                la_params_wd.append(state["la_param"])
                kahan_comps_wd.append(state["kahan_comp"])
            else:
                params_no_wd.append(p)
                grads_no_wd.append(p.grad)
                exp_avgs_no_wd.append(state["exp_avg"])
                exp_avg_sqs_no_wd.append(state["exp_avg_sq"])
                la_params_no_wd.append(state["la_param"])
                kahan_comps_no_wd.append(state["kahan_comp"])

        if not group["setup"]:
            group["setup"] = True
            group["step"] = torch.tensor(0, dtype=torch.int32)

            if group["foreach"] is None:
                _, group["foreach"] = _default_to_fused_or_foreach(params_wd + params_no_wd, False, False)

    @torch.no_grad()
    def step(self, closure: Callable | None = None, param: Tensor | None = None):
        loss = None
        if closure is not None and param is None:
            with torch.enable_grad():
                loss = closure()

        if param is None:
            for group in self.param_groups:
                params_wd, params_no_wd, grads_wd, grads_no_wd = [], [], [], []
                exp_avgs_wd, exp_avgs_no_wd, exp_avg_sqs_wd, exp_avg_sqs_no_wd = [], [], [], []
                la_params_wd, la_params_no_wd, kahan_comps_wd, kahan_comps_no_wd = [], [], [], []

                self._init_group(
                    group,
                    params_wd,
                    params_no_wd,
                    grads_wd,
                    grads_no_wd,
                    exp_avgs_wd,
                    exp_avgs_no_wd,
                    exp_avg_sqs_wd,
                    exp_avg_sqs_no_wd,
                    la_params_wd,
                    la_params_no_wd,
                    kahan_comps_wd,
                    kahan_comps_no_wd
                )

                # Call ranger for parameters with weight decay
                if len(params_wd) > 0:
                    ranger(
                        params=params_wd,
                        grads=grads_wd,
                        exp_avgs=exp_avgs_wd,
                        exp_avg_sqs=exp_avg_sqs_wd,
                        la_params=la_params_wd,
                        kahan_comps=kahan_comps_wd,
                        lr=group["lr"],
                        beta1=group["mom"],
                        beta2=group["sqr_mom"],
                        weight_decay=group["wd"],
                        eps=group["eps"],
                        k=group["k"],
                        alpha=group["alpha"],
                        step=group["step"],
                        decouple_wd=group["decouple_wd"],
                        decouple_lr=group["decouple_lr"],
                        max_lr=group["max_lr"],
                        kahan_sum=group["kahan_sum"],
                        foreach=group["foreach"],
                        gradient_release=False,
                        optimizer_accumulation=False,
                    )

                # Call ranger for parameters without weight decay
                if len(params_no_wd) > 0:
                    ranger(
                        params=params_no_wd,
                        grads=grads_no_wd,
                        exp_avgs=exp_avgs_no_wd,
                        exp_avg_sqs=exp_avg_sqs_no_wd,
                        la_params=la_params_no_wd,
                        kahan_comps=kahan_comps_no_wd,
                        lr=group["lr"],
                        beta1=group["mom"],
                        beta2=group["sqr_mom"],
                        weight_decay=0,
                        eps=group["eps"],
                        k=group["k"],
                        alpha=group["alpha"],
                        step=group["step"],
                        decouple_wd=group["decouple_wd"],
                        decouple_lr=group["decouple_lr"],
                        max_lr=group["max_lr"],
                        kahan_sum=group["kahan_sum"],
                        foreach=group["foreach"],
                        gradient_release=False,
                        optimizer_accumulation=False,
                    )
        else:
            state = self.state[param]
            group = state["group"]
            self._init_state(group, state, param)

            ranger(
                params=param,
                grads=param.grad,
                exp_avgs=state["exp_avg"],
                exp_avg_sqs=state["exp_avg_sq"],
                la_params=state["la_param"],
                kahan_comps=state["kahan_comp"],
                lr=group["lr"],
                beta1=group["mom"],
                beta2=group["sqr_mom"],
                weight_decay=group["wd"] if state.get("do_wd", True) else 0,
                eps=group["eps"],
                k=group["k"],
                alpha=group["alpha"],
                step=state["step"],
                decouple_wd=group["decouple_wd"],
                decouple_lr=group["decouple_lr"],
                max_lr=group["max_lr"],
                kahan_sum=group["kahan_sum"],
                foreach=False,
                gradient_release=True,
                optimizer_accumulation=self._optimizer_accumulation,
            )

        return loss

In [ ]:
show_doc(RangerOptimiOptimizer)

## SGD -

In [ ]:
#|exporti
class SGDOptimiOptimizer(SGD, FastaiOptimizerAdapter):
    """SGD optimizer. Optionally with momentum and decoupled weight decay.

    Parameters
    ----------
    params
        Iterable of parameters to optimize or dicts defining parameter groups
    lr
        Learning rate
    momentum
        Momentum factor. Gradient moving average coefficient if `dampening` is True (default: 0)
    weight_decay
        Weight decay coefficient. If `decouple_wd` and `decouple_lr` are False, applies L2 penalty (default: 0)
    dampening
        Use dampening for momentum update (default: False)
    decouple_wd
        Apply decoupled weight decay instead of L2 penalty (default: False)
    decouple_lr
        Apply fully decoupled weight decay instead of L2 penalty (default: False)
    max_lr
        Maximum scheduled learning rate. Set if `lr` is not the maximum scheduled learning rate and `decouple_lr` is True (default: None)
    torch_init
        Initialize momentum buffer with first gradient instead of zeroes. Enable to match PyTorch SGD when using dampening (default: False)
    kahan_sum
        Enables Kahan summation for more accurate parameter updates when training in low precision (float16 or bfloat16). If unspecified, automatically applies for low precision parameters (default: None)
    foreach
        Enables the foreach implementation. If unspecified, tries to use foreach over for-loop implementation since it is significantly faster (default: None)
    """

    def __init__(
        self,
        params: Iterable[Tensor] | Iterable[dict],
        lr: float,
        mom: float = 0,
        wd: float = 0,
        dampening: bool = False,
        decouple_wd: bool = False,
        decouple_lr: bool = False,
        max_lr: float | None = None,
        torch_init: bool = False,
        kahan_sum: bool | None = None,
        foreach: bool | None = None,
    ):
        params = super().convert_params(params)

        if not 0.0 <= wd:
            raise ValueError(f"Invalid weight decay: {wd=}")

        defaults = dict(
            lr=lr,
            mom=mom,
            wd=wd,
            dampening=dampening,
            weight_decay=wd,
            decouple_wd=decouple_wd,
            decouple_lr=decouple_lr,
            max_lr=max_lr,
            torch_init=torch_init,
            kahan_sum=kahan_sum,
            foreach=foreach,
            gradient_release=False,
            setup=False,
        )
        super(SGD, self).__init__(params, defaults)

    def _init_group(
        self,
        group: dict[str, Any],
        params_wd: list[Tensor],
        params_no_wd: list[Tensor],
        grads_wd: list[Tensor],
        grads_no_wd: list[Tensor],
        exp_avgs_wd: list[Tensor],
        exp_avgs_no_wd: list[Tensor],
        kahan_comps_wd: list[Tensor],
        kahan_comps_no_wd: list[Tensor],
    ):
        for p in group["params"]:
            if p.grad is None:
                continue

            state = self.state[p]
            self._init_state(group, state, p)

            if state.get("do_wd", True):
                params_wd.append(p)
                grads_wd.append(p.grad)
                exp_avgs_wd.append(state["exp_avg"])
                kahan_comps_wd.append(state["kahan_comp"])
            else:
                params_no_wd.append(p)
                grads_no_wd.append(p.grad)
                exp_avgs_no_wd.append(state["exp_avg"])
                kahan_comps_no_wd.append(state["kahan_comp"])

        if not group["setup"]:
            group["setup"] = True

            if group["foreach"] is None:
                _, group["foreach"] = _default_to_fused_or_foreach(params_wd + params_no_wd, False, False)

    @torch.no_grad()
    def step(self, closure: Callable | None = None, param: Tensor | None = None):
        loss = None
        if closure is not None and param is None:
            with torch.enable_grad():
                loss = closure()

        if param is None:
            for group in self.param_groups:
                params_wd, params_no_wd, grads_wd, grads_no_wd = [], [], [], []
                exp_avgs_wd, exp_avgs_no_wd, kahan_comps_wd, kahan_comps_no_wd = [], [], [], []

                self._init_group(
                    group,
                    params_wd,
                    params_no_wd,
                    grads_wd,
                    grads_no_wd,
                    exp_avgs_wd,
                    exp_avgs_no_wd,
                    kahan_comps_wd,
                    kahan_comps_no_wd
                )

                # Call sgd for parameters with weight decay
                if len(params_wd) > 0:
                    sgd(
                        params=params_wd,
                        grads=grads_wd,
                        exp_avgs=exp_avgs_wd,
                        kahan_comps=kahan_comps_wd,
                        lr=group["lr"],
                        momentum=group["mom"],
                        weight_decay=group["wd"],
                        dampening=group["dampening"],
                        decouple_wd=group["decouple_wd"],
                        decouple_lr=group["decouple_lr"],
                        max_lr=group["max_lr"],
                        kahan_sum=group["kahan_sum"],
                        foreach=group["foreach"],
                        gradient_release=False,
                        optimizer_accumulation=False,
                    )

                # Call sgd for parameters without weight decay
                if len(params_no_wd) > 0:
                    sgd(
                        params=params_no_wd,
                        grads=grads_no_wd,
                        exp_avgs=exp_avgs_no_wd,
                        kahan_comps=kahan_comps_no_wd,
                        lr=group["lr"],
                        momentum=group["mom"],
                        weight_decay=0,
                        dampening=group["dampening"],
                        decouple_wd=group["decouple_wd"],
                        decouple_lr=group["decouple_lr"],
                        max_lr=group["max_lr"],
                        kahan_sum=group["kahan_sum"],
                        foreach=group["foreach"],
                        gradient_release=False,
                        optimizer_accumulation=False,
                    )
        else:
            state = self.state[param]
            group = state["group"]
            self._init_state(group, state, param)

            sgd(
                params=param,
                grads=param.grad,
                exp_avgs=state["exp_avg"],
                kahan_comps=state["kahan_comp"],
                lr=group["lr"],
                momentum=group["mom"],
                weight_decay=group["wd"] if state.get("do_wd", True) else 0,
                dampening=group["dampening"],
                decouple_wd=group["decouple_wd"],
                decouple_lr=group["decouple_lr"],
                max_lr=group["max_lr"],
                kahan_sum=group["kahan_sum"],
                foreach=False,
                gradient_release=True,
                optimizer_accumulation=self._optimizer_accumulation,
            )

        return loss

In [ ]:
show_doc(SGDOptimiOptimizer)

## StableAdamW -

In [ ]:
#|exporti
class StableAdamWOptimiOptimizer(StableAdamW, FastaiOptimizerAdapter):
    """StableAdamW optimizer. An AdamW-Adafactor hybrid with learning rate update clipping.

    Parameters
    ----------
    params
        Iterable of parameters to optimize or dicts defining parameter groups
    lr
        Learning rate
    betas
        Coefficients for gradient and squared gradient moving averages (default: (0.9, 0.99))
    weight_decay
        Weight decay coefficient. If `decouple_lr` is False, applies decoupled weight decay (default: 1e-2)
    eps
        Added to denominator to improve numerical stability (default: 1e-6)
    decouple_lr
        Apply fully decoupled weight decay instead of decoupled weight decay (default: False)
    max_lr
        Maximum scheduled learning rate. Set if `lr` is not the maximum scheduled learning rate and `decouple_lr` is True (default: None)
    kahan_sum
        Enables Kahan summation for more accurate parameter updates when training in low precision (float16 or bfloat16). If unspecified, automatically applies for low precision parameters (default: None)
    foreach
        Enables the foreach implementation. If unspecified, tries to use foreach over for-loop implementation since it is significantly faster (default: None)
    """

    def __init__(
        self,
        params: Iterable[Tensor] | Iterable[dict],
        lr: float,
        mom: float = 0.9,
        sqr_mom: float = 0.99,
        wd: float = 1e-2,
        eps: float = 1e-6,
        decouple_lr: bool = False,
        max_lr: float | None = None,
        kahan_sum: bool | None = None,
        foreach: bool | None = None,
    ):
        params = super().convert_params(params)

        if not 0.0 <= mom < 1.0:
            raise ValueError(f"Invalid momentum parameter: {mom=}")
        if not 0.0 <= sqr_mom< 1.0:
            raise ValueError(f"Invalid squared momentum parameter: {sqr_mom=}")
        if not 0.0 <= wd:
            raise ValueError(f"Invalid weight decay: {wd=}")
        if not 0.0 <= eps:
            raise ValueError(f"Invalid epsilon: {eps=}")

        defaults = dict(
            lr=lr,
            mom=mom,
            sqr_mom=sqr_mom,
            wd=wd,
            eps=eps,
            weight_decay=wd,
            decouple_lr=decouple_lr,
            max_lr=max_lr,
            kahan_sum=kahan_sum,
            foreach=foreach,
            gradient_release=False,
            setup=False,
        )
        super(StableAdamW, self).__init__(params, defaults)

    def _init_group(
        self,
        group: dict[str, Any],
        params_wd: list[Tensor],
        params_no_wd: list[Tensor],
        grads_wd: list[Tensor],
        grads_no_wd: list[Tensor],
        exp_avgs_wd: list[Tensor],
        exp_avgs_no_wd: list[Tensor],
        exp_avg_sqs_wd: list[Tensor],
        exp_avg_sqs_no_wd: list[Tensor],
        eps_sqs_wd: list[Tensor],
        eps_sqs_no_wd: list[Tensor],
        kahan_comps_wd: list[Tensor],
        kahan_comps_no_wd: list[Tensor],
    ):
        for p in group["params"]:
            if p.grad is None:
                continue

            state = self.state[p]
            self._init_state(group, state, p)

            if state.get("do_wd", True):
                params_wd.append(p)
                grads_wd.append(p.grad)
                exp_avgs_wd.append(state["exp_avg"])
                exp_avg_sqs_wd.append(state["exp_avg_sq"])
                eps_sqs_wd.append(state["eps_sq"])
                kahan_comps_wd.append(state["kahan_comp"])
            else:
                params_no_wd.append(p)
                grads_no_wd.append(p.grad)
                exp_avgs_no_wd.append(state["exp_avg"])
                exp_avg_sqs_no_wd.append(state["exp_avg_sq"])
                eps_sqs_no_wd.append(state["eps_sq"])
                kahan_comps_no_wd.append(state["kahan_comp"])

        if not group["setup"]:
            group["setup"] = True
            group["step"] = torch.tensor(0, dtype=torch.int32)

            if group["foreach"] is None:
                _, group["foreach"] = _default_to_fused_or_foreach(params_wd + params_no_wd, False, False)

    @torch.no_grad()
    def step(self, closure: Callable | None = None, param: Tensor | None = None):
        loss = None
        if closure is not None and param is None:
            with torch.enable_grad():
                loss = closure()

        if param is None:
            for group in self.param_groups:
                params_wd, params_no_wd, grads_wd, grads_no_wd = [], [], [], []
                exp_avgs_wd, exp_avgs_no_wd, exp_avg_sqs_wd, exp_avg_sqs_no_wd = [], [], [], []
                eps_sqs_wd, eps_sqs_no_wd, kahan_comps_wd, kahan_comps_no_wd = [], [], [], []

                self._init_group(
                    group,
                    params_wd,
                    params_no_wd,
                    grads_wd,
                    grads_no_wd,
                    exp_avgs_wd,
                    exp_avgs_no_wd,
                    exp_avg_sqs_wd,
                    exp_avg_sqs_no_wd,
                    eps_sqs_wd,
                    eps_sqs_no_wd,
                    kahan_comps_wd,
                    kahan_comps_no_wd
                )

                # Call stableadamw for parameters with weight decay
                if len(params_wd) > 0:
                    stableadamw(
                        params=params_wd,
                        grads=grads_wd,
                        exp_avgs=exp_avgs_wd,
                        exp_avg_sqs=exp_avg_sqs_wd,
                        eps_sqs=eps_sqs_wd,
                        kahan_comps=kahan_comps_wd,
                        lr=group["lr"],
                        beta1=group["mom"],
                        beta2=group["sqr_mom"],
                        weight_decay=group["wd"],
                        eps=group["eps"],
                        step=group["step"],
                        decouple_lr=group["decouple_lr"],
                        max_lr=group["max_lr"],
                        kahan_sum=group["kahan_sum"],
                        foreach=group["foreach"],
                        gradient_release=False,
                        optimizer_accumulation=False,
                    )

                # Call stableadamw for parameters without weight decay
                if len(params_no_wd) > 0:
                    stableadamw(
                        params=params_no_wd,
                        grads=grads_no_wd,
                        exp_avgs=exp_avgs_no_wd,
                        exp_avg_sqs=exp_avg_sqs_no_wd,
                        eps_sqs=eps_sqs_no_wd,
                        kahan_comps=kahan_comps_no_wd,
                        lr=group["lr"],
                        beta1=group["mom"],
                        beta2=group["sqr_mom"],
                        weight_decay=0,
                        eps=group["eps"],
                        step=group["step"],
                        decouple_lr=group["decouple_lr"],
                        max_lr=group["max_lr"],
                        kahan_sum=group["kahan_sum"],
                        foreach=group["foreach"],
                        gradient_release=False,
                        optimizer_accumulation=False,
                    )
        else:
            state = self.state[param]
            group = state["group"]
            self._init_state(group, state, param)

            stableadamw(
                params=param,
                grads=param.grad,
                exp_avgs=state["exp_avg"],
                exp_avg_sqs=state["exp_avg_sq"],
                eps_sqs=state["eps_sq"],
                kahan_comps=state["kahan_comp"],
                lr=group["lr"],
                beta1=group["mom"],
                beta2=group["sqr_mom"],
                weight_decay=group["wd"] if state.get("do_wd", True) else 0,
                eps=group["eps"],
                step=state["step"],
                decouple_lr=group["decouple_lr"],
                max_lr=group["max_lr"],
                kahan_sum=group["kahan_sum"],
                foreach=False,
                gradient_release=True,
                optimizer_accumulation=self._optimizer_accumulation,
            )

        return loss

In [ ]:
show_doc(StableAdamWOptimiOptimizer)

## fastai Compatiblity Tests -

In [ ]:
#|hide
from fastxtend.test_utils import *

In [ ]:
#|hide
params = [tst_params(), tst_params(), tst_params()]
for opt in [SGDOptimiOptimizer(params, lr=0.1), AdamWOptimiOptimizer(params, lr=0.1)]:
    #Freezing the first layer
    opt.freeze_to(1)
    req_grad = Self.requires_grad()
    test_eq(L(params[0]).map(req_grad), [False]*4)
    for i in {1,2}:
        test_eq(L(params[i]).map(req_grad), [True]*4)

    #Unfreezing
    opt.unfreeze()
    for i in range(2):
        test_eq(L(params[i]).map(req_grad), [True]*4)

In [ ]:
#|hide
params = [tst_params(), tst_params(), tst_params()]
for opt in [AdamOptimiOptimizer(params, lr=0.1), LionOptimiOptimizer(params, lr=0.1)]:
    for p in L(params[1])[[1,3]]:
        opt.state[p] = {'force_train': True}
    opt.freeze()
    test_eq(L(params[0]).map(req_grad), [False]*4)
    test_eq(L(params[1]).map(req_grad), [False, True, False, True])
    test_eq(L(params[2]).map(req_grad), [True]*4)

In [ ]:
#|hide
params = [tst_params(), tst_params(), tst_params()]
for opt in [RAdamOptimiOptimizer(params, lr=1e-3), RangerOptimiOptimizer(params, lr=1e-3)]:
    opt.set_hypers(lr=3e-3, mom=0.98, sqr_mom=0.9999, eps=1e-9, wd=0.01)
    for opt_dict in opt.hypers:
        assert opt_dict.items() >= dict(lr=3e-3, mom=0.98, sqr_mom=0.9999, eps=1e-9, wd=0.01).items()

## Optimi Optimizer Tests -

In [ ]:
#|hide
#|cuda
from fastai.optimizer import (weight_decay, l2_reg, average_grad, momentum_step,
                              average_sqr_grad,step_stat, adam_step, radam_step, Lookahead, Optimizer)

In [ ]:
#|hide
#|cuda
def assert_most_approx_close(a, b, rtol=1e-3, atol=1e-3, max_error_count=10):
    idx = torch.isclose(a, b, rtol=rtol, atol=atol)
    error_count = (idx == 0).sum().item()
    if error_count > max_error_count:
        print(f"Too many values not close: assert {error_count} < {max_error_count}")
        torch.testing.assert_close(a, b, rtol=rtol, atol=atol)

def tst_param(shape):
    "Create a tensor with `val` and a gradient of `grad` for testing"
    param = torch.rand(shape, device='cuda', dtype=torch.float32) * 0.1
    param.grad = torch.rand(shape, device='cuda', dtype=param.dtype) * 0.01
    return param

def tst_params():
    params1, params2 = [], []
    for i in range(4):
        param1 = tst_param(4096)
        param2 = param1.clone()
        param2.grad = param1.grad.clone()
        params1.append(param1)
        params2.append(param2)
    return params1, params2

In [ ]:
#|hide
#|cuda
def test_adam(
    params:Listified[Tensor], # Model parameters or parameter groups
    lr:float, # Default learning rate
    mom:float=0.9, # Gradient moving average (β1) coefficient
    sqr_mom:float=0.99, # Gradient squared moving average (β2) coefficient
    eps:float=1e-6, # Added for numerical stability
    wd:float=0.01, # Optional weight decay (true or L2)
    optimi:bool=False, # Use fused optimi implementation
) -> Optimizer|AdamOptimiOptimizer:
    if optimi:
        return AdamOptimiOptimizer(params, lr=lr, mom=mom, sqr_mom=sqr_mom, eps=eps, wd=wd)
    else:
        cbs = [weight_decay, partial(average_grad, dampening=True), average_sqr_grad, step_stat, adam_step]
        return Optimizer(params, cbs, lr=lr, mom=mom, sqr_mom=sqr_mom, eps=eps, wd=wd)

In [ ]:
#|hide
#|cuda
params_org, params_optimi = tst_params()
for p, e in zip(params_org, params_optimi):
    assert_most_approx_close(p, e)

opt_org = test_adam(params_org, lr=1e-3)
opt_optimi = test_adam(params_optimi, lr=1e-3, optimi=True)

for i in range(10):
    opt_org.step()
    opt_optimi.step()

for p, e in zip(params_org, params_optimi):
    assert_most_approx_close(p, e)

opt_org.step()
opt_optimi.step()
for p, e in zip(params_org, params_optimi):
    assert_most_approx_close(p, e)

In [ ]:
def test_radam(
    params:Listified[Tensor], # Model parameters or parameter groups
    lr:float, # Default learning rate
    mom:float=0.9, # Gradient moving average (β1) coefficient
    sqr_mom:float=0.99, # Gradient squared moving average (β2) coefficient
    eps:float=1e-6, # Added for numerical stability
    wd:float=0.01, # Optional weight decay (true or L2)
    beta:float=0., # Set to enable SAdam
    optimi:bool=False, # Use fused optimi implementation
) -> Optimizer|RAdamOptimiOptimizer:
    if optimi:
        return RAdamOptimiOptimizer(params, lr=lr, mom=mom, sqr_mom=sqr_mom, eps=eps, wd=wd)
    else:
        cbs = [weight_decay, partial(average_grad, dampening=True), average_sqr_grad, step_stat, radam_step]
        return Optimizer(params, cbs, lr=lr, mom=mom, sqr_mom=sqr_mom, eps=eps, wd=wd, beta=beta)

In [ ]:
#|hide
#|cuda
params_org, params_optimi = tst_params()
for p, e in zip(params_org, params_optimi):
    assert_most_approx_close(p, e)

opt_org = test_radam(params_org, lr=1e-3)

opt_optimi = test_radam(params_optimi, lr=1e-3, optimi=True)

for i in range(10):
    opt_org.step()
    opt_optimi.step()

for p, e in zip(params_org, params_optimi):
    assert_most_approx_close(p, e)

opt_org.step()
opt_optimi.step()
for p, e in zip(params_org, params_optimi):
    assert_most_approx_close(p, e)

In [ ]:
def test_ranger(
    params:Listified[Tensor], # Model parameters or parameter groups
    lr:float, # Default learning rate
    mom:float=0.9, # Gradient moving average (β1) coefficient
    sqr_mom:float=0.99, # Gradient squared moving average (β2) coefficient
    eps:float=1e-6, # Added for numerical stability
    wd:float=0.01, # Optional weight decay (true or L2)
    k:int=6, # How often to conduct Lookahead step
    alpha:float=0.5, # Slow weight moving average coefficient
    optimi:bool=False, # Use fused optimi implementation
) -> Lookahead|RangerOptimiOptimizer:
    if optimi:
        return RangerOptimiOptimizer(params, lr=lr, mom=mom, sqr_mom=sqr_mom, eps=eps, wd=wd)
    else:
        return Lookahead(test_radam(params, lr=lr, mom=mom, sqr_mom=sqr_mom, wd=wd, eps=eps), k=k, alpha=alpha)

In [ ]:
#|hide
#|cuda
params_org, params_optimi = tst_params()
for p, e in zip(params_org, params_optimi):
    assert_most_approx_close(p, e)

opt_org = test_ranger(params_org, lr=1e-3)
opt_optimi = test_ranger(params_optimi, lr=1e-3, optimi=True)

for i in range(10):
    opt_org.step()
    opt_optimi.step()

for p, e in zip(params_org, params_optimi):
    assert_most_approx_close(p, e)

opt_org.step()
opt_optimi.step()
for p, e in zip(params_org, params_optimi):
    assert_most_approx_close(p, e)

In [ ]:
#|hide
#|cuda
def test_sgd(
    params:Listified[Tensor], # Model parameters or parameter groups
    lr:float, # Default learning rate
    mom:float=0.9, # Gradient moving average (β1) coefficient
    wd:float=0., # Optional L2 weight decay
    optimi:bool=False, # Use fused optimi implementation
) -> Optimizer|SGDOptimiOptimizer:
    if optimi:
        return SGDOptimiOptimizer(params, lr=lr, mom=mom, wd=wd)
    else:
        cbs = [l2_reg, average_grad, momentum_step]
        return Optimizer(params, cbs, lr=lr, mom=mom, wd=wd)

In [ ]:
#|hide
#|cuda
params_org, params_optimi = tst_params()
for p, e in zip(params_org, params_optimi):
    assert_most_approx_close(p, e)

opt_org = test_sgd(params_org, lr=0.01)
opt_optimi = test_sgd(params_optimi, lr=0.01, optimi=True)

for i in range(10):
    opt_org.step()
    opt_optimi.step()

for p, e in zip(params_org, params_optimi):
    assert_most_approx_close(p, e)

opt_org.step()
opt_optimi.step()
for p, e in zip(params_org, params_optimi):
    assert_most_approx_close(p, e)